# Convert annotated point cloud to a GeoTiff raster

We create a raster with two bands:
1. The number of tree-points per cell
2. The average height above ground (HAG) for that cell

In [ ]:
import set_path

import numpy as np
import laspy
import pathlib
from tqdm.notebook import tqdm

from upcp.utils import clip_utils

import gvl.helper_functions as helpers

## Settings

In [ ]:
BASE_FOLDER = pathlib.Path('../datasets')

input_dir = BASE_FOLDER / 'AHN4' / 'AMS_subtiles_1000_pred'
output_dir = BASE_FOLDER / 'GeoTiff_subtiles_1000'

EPSG = 28992     # RD coordinates
grid_res = 0.5   # 8cm resolution grid

include_overlap = True  # Add the existing overlap buffer aropund the tile
BUFFER = 5

# Our classification
UNKNOWN = 0
TREE = 1
NOISE = 2
OTHER = 0

In [ ]:
# Create output folder
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

## Helper functions

In [ ]:
from scipy.stats import binned_statistic_2d

def get_tile_grid(points, hag, x_min, y_min, tilewidth, res):
    x_edge = np.arange(x_min, x_min+tilewidth+res, res)
    y_edge = np.arange(y_min, y_min+tilewidth+res, res)
    n_points = binned_statistic_2d(x=points[:, 0], 
                                   y=points[:, 1],
                                   values=None,
                                   bins=[x_edge, y_edge],
                                   statistic='count').statistic
    mean_hag = binned_statistic_2d(x=points[:, 0], 
                                   y=points[:, 1],
                                   values=hag,
                                   bins=[x_edge, y_edge],
                                   statistic='mean').statistic
    return n_points, mean_hag


from osgeo import gdal
from osgeo import osr

def save_geotiff(fname, n_points, mean_hag, x_min, y_min, res):
    x_dim, y_dim = n_points.shape

    # set geotransform
    geotransform = (x_min, res, 0, y_min, 0, res)

    # create the 3-band raster file
    geotiff = gdal.GetDriverByName('GTiff').Create(fname, x_dim, y_dim, 2, gdal.GDT_Float32)

    geotiff.SetGeoTransform(geotransform)    # specify coords
    srs = osr.SpatialReference()             # establish encoding
    srs.ImportFromEPSG(EPSG)                 # set coordinate system
    geotiff.SetProjection(srs.ExportToWkt()) # export coords to file

    geotiff.GetRasterBand(1).WriteArray(n_points)
    geotiff.GetRasterBand(1).SetNoDataValue(0)
    geotiff.GetRasterBand(1).SetDescription('n_points')

    geotiff.GetRasterBand(2).WriteArray(mean_hag)
    geotiff.GetRasterBand(2).SetNoDataValue(0)
    geotiff.GetRasterBand(2).SetDescription('hag')

    geotiff.FlushCache()                     # write to disk
    geotiff = None

## Main loop

In [ ]:
input_files = list(pathlib.Path(input_dir).glob('merged*.laz'))

In [ ]:
# Check existing output files (ignore this cell to re-run for all tiles)
existing_files = list(pathlib.Path(output_dir).glob('raster*.tif'))
existing_codes = {helpers.get_tilecode_from_filename(file.name)
                  for file in existing_files}

input_files = [file for file in input_files
               if helpers.get_tilecode_from_filename(file.name) not in existing_codes]

In [ ]:
pbar = tqdm(input_files, unit='file', smoothing=0)

for file in pbar:
    tilecode = helpers.get_tilecode_from_filename(file.name)
    pbar.set_postfix_str(tilecode)
    
    # Load LAS data
    las = laspy.read(file)
    mask = las.label == TREE
    points_xyz = np.vstack((las.x[mask], las.y[mask], las.z[mask])).T
    hag = las.hag[mask]
    
    if include_overlap:
        ((x_min, y_max), (x_max, y_min)) = helpers.get_bbox_from_tile_code(tilecode, padding=BUFFER)
        tilewidth = x_max - x_min  # Assume square tile
    else:
        ((x_min, y_max), (x_max, y_min)) = helpers.get_bbox_from_tile_code(tilecode, padding=0)
        tilewidth = x_max - x_min  # Assume square tile
        mask = clip_utils.rectangle_clip(points_xyz, (x_min, y_min, x_max, y_max))
        points_xyz = points_xyz[mask, :]
        hag = hag[mask]

    n_points, mean_hag = get_tile_grid(points_xyz, hag, x_min, y_min, tilewidth, grid_res)
    
    output_file = pathlib.Path(output_dir) / f'raster_{tilecode}.tif'
    save_geotiff(output_file.as_posix(), n_points.T, mean_hag.T, x_min, y_min, grid_res)